[Reference](https://pub.towardsai.net/real-time-stock-news-sentiment-analyzer-54eaa91c5634)

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen, Request

In [2]:
def get_ticks(url):
    req = Request(url=url,headers={"User-Agent": "Chrome"}) 
    response = urlopen(req)    
    html = BeautifulSoup(response,"html.parser")
    ticks_table = html.find(class_ = 'page')
    
    ticks = list()
    stocks = list()
    for name_box in ticks_table.find_all('a', href = True):
        stocks.append(name_box.text.strip())
        ticks.append(name_box['href'].split('-')[-1].strip())
    d = {'stock':stocks, 'tick':ticks}
    df = pd.DataFrame(data = d)
    return df

In [5]:
url = "https://www.tickertape.in/stocks"
tick_df = get_ticks(url)
tick_df.head()

,stock,tick
0,Adani Enterprises Ltd,ADEL
1,Adani Green Energy Ltd,ADNA
2,Adani Ports and Special Economic Zone Ltd,APSE
3,Adani Power Ltd,ADAN
4,Adani Total Gas Ltd,ADAG


In [3]:
def get_data(url):
    req = Request(url=url,headers={"User-Agent": "Chrome"}) 
    response = urlopen(req).read()    
    html = BeautifulSoup(response,"html.parser")
    news_table = html.find(class_ = 'latest-news-holder')
    
    news = list()
    
    for name_box in news_table.find_all('p', class_='shave-root'):
        news.append(name_box.text.strip())
    
    return news

In [6]:
news = list()
for i, tick in enumerate(tick_df['tick']):
    url = "https://www.tickertape.in/stocks/" + tick + "/news?checklist=basic&ref=stock-overview_overview-sections&type=news"
    headlines = get_data(url)
    news.append(headlines)

In [7]:
for i, n in enumerate(news):
    print(i+1, ")", n)
    print('-------')

1 ) ['Adani Enterprises announces incorporation of step-down subsidiary', 'Adani Enterprises rallies 5%, hits record high; stock soars 27% in a month', 'Stocks that will see action on April 18']
-------
2 ) ['Taking Stock | Market tanks for the fourth day; IT, banks hammered', 'Taking Stock | Market cheers RBI policy, Sensex gains 412 points, Nifty above 17,700', 'M-cap of 7 of top 10 firms tumbles over Rs 1.32 lakh cr; Reliance biggest drag']
-------
3 ) ['Adani Ports buys Ocean Sparkle for Rs 1700 cr through Adani Harbour Svces', 'Adani Ports to acquire marine services firm Ocean Sparkle', 'Taking Stock | Sensex down 575 points, Nifty below 17,700 ahead of RBI policy']
-------
4 ) ['Adani Power rallies 100% in a month to enter 50 most valued cos’ club', 'Adani Power Shares Surge 109% in a Month. Should You Buy, Hold or Sell?', 'Adani Power enters top-50 most-valued firms club; zooms 109% in a month']
-------
5 ) []
-------
6 ) ['Adani Transmission Ltd Spikes 4.45%, S&P BSE Power inde

# Sentiment Analysis of Financial News


In [9]:
pip install transformers

     |████████████████████████████████| 4.0 MB 22.3 MB/s 
     |████████████████████████████████| 77 kB 8.0 MB/s 
     |████████████████████████████████| 6.6 MB 49.9 MB/s 
     |████████████████████████████████| 895 kB 52.3 MB/s 
     |████████████████████████████████| 596 kB 53.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [10]:
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

Downloading:   0%|          | 0.00/533 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/221k [00:00<?, ?B/s]

In [11]:
labels = {0:'neutral', 1:'positive',2:'negative'}

In [12]:
def detect(news):
    tot_val = list()
    for n in news:
        if len(n) == 0:
            tot_val.append(['neutral'])
        else:
            inputs = tokenizer(n, return_tensors="pt", padding=True)
            outputs = finbert(**inputs)[0]
            val = list()
            for idx, sent in enumerate(n):
                val.append(labels[np.argmax(outputs.detach().numpy()[idx])])
                print(sent, '----', val[idx])
            print('#######################################################')    
            tot_val.append(val)
    return tot_val

tot_val = detect(news)
print(tot_val)

Adani Enterprises announces incorporation of step-down subsidiary ---- neutral
Adani Enterprises rallies 5%, hits record high; stock soars 27% in a month ---- positive
Stocks that will see action on April 18 ---- neutral
#######################################################
Taking Stock | Market tanks for the fourth day; IT, banks hammered ---- negative
Taking Stock | Market cheers RBI policy, Sensex gains 412 points, Nifty above 17,700 ---- positive
M-cap of 7 of top 10 firms tumbles over Rs 1.32 lakh cr; Reliance biggest drag ---- negative
#######################################################
Adani Ports buys Ocean Sparkle for Rs 1700 cr through Adani Harbour Svces ---- neutral
Adani Ports to acquire marine services firm Ocean Sparkle ---- neutral
Taking Stock | Sensex down 575 points, Nifty below 17,700 ahead of RBI policy ---- negative
#######################################################
Adani Power rallies 100% in a month to enter 50 most valued cos’ club ---- neutral
Adani

In [13]:
def get_sent(val):
    agg = 0
    for i in val:
        if i == 'positive':
            agg = agg + 1
        elif i == 'negative':
            agg = agg - 1

    if agg > 0:
        return('positive')
    elif agg < 0:
        return('negative')
    else:
        return('neutral')

In [14]:
sent = list()
for i in tot_val:
    sent.append(get_sent(i))
    
tick_df['sentiment'] = sent 
tick_df.head(20)

,stock,tick,sentiment
0,Adani Enterprises Ltd,ADEL,positive
1,Adani Green Energy Ltd,ADNA,negative
2,Adani Ports and Special Economic Zone Ltd,APSE,negative
3,Adani Power Ltd,ADAN,positive
4,Adani Total Gas Ltd,ADAG,neutral
5,Adani Transmission Ltd,ADAI,neutral
6,Adani Wilmar Ltd,AWL,neutral
7,Ambuja Cements Ltd,ABUJ,positive
8,Apollo Hospitals Enterprise Ltd,APLH,positive
9,Asian Paints Ltd,ASPN,positive
